<a href="https://colab.research.google.com/github/Jaywestty/News-Crime-Classification/blob/main/Text_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **NEWS TEXT SUMMARIZER PROJECT**

####**Project Description:**
This project aims to automatically summarize news articles into concise, factual highlights using Hugging Face Transformers. The summarization model is based on the bart-base architecture, chosen for its strong performance on abstractive summarization while remaining lightweight enough to run within Google Colab's free-tier resource limits. The dataset, sourced from Hugging Face’s public datasets repository, contains diverse news articles for training and evaluation. The system is designed to generate short, accurate, and easily readable summaries that retain the key points of the original article, making it useful for quick news consumption.

#### **Install dependecies**

In [ ]:
!pip install transformers datasets evaluate rouge_score accelerate nltk -q

#### **Import required libraries**

In [ ]:
from datasets import load_dataset
from transformers import BartForConditionalGeneration, BartTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import numpy as np
import torch
import nltk
import gc
import evaluate
nltk.download('punkt')

In [ ]:
#Clear up memory to aid colab limit
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

#### **Load Bart tokenizer and model**

In [ ]:
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

print(f"Model loaded! Parameters: {model.num_parameters():,}")
clear_memory()

#### **Load Dataset**

In [ ]:
print("Loading CNN-DailyMail dataset...")
dataset = load_dataset('cnn_dailymail', '3.0.0')

print("Sample article:\n", dataset['train'][0]['article'][:200])
print("\nSample summary:\n", dataset['train'][0]['highlights'])

# Reduce dataset for Colab constraints
train_dataset = dataset['train'].select(range(8000))  # Slightly smaller for BART
val_dataset = dataset['validation'].select(range(800))
test_dataset = dataset['test'].select(range(800))

print(f"Dataset sizes - Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

#### **BART-specific preprocessing**

In [ ]:
max_input_length = 1024  # BART can handle longer inputs
max_target_length = 142  # CNN-DM standard summary length

def preprocess(example):
    model_inputs = tokenizer(
        example['article'],
        max_length=max_input_length,
        truncation=True,
        padding="max_length"  # Changed from True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example['highlights'],
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )

    # Replace pad token id with -100 for label loss masking
    labels_ids = labels["input_ids"]
    labels_ids = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels_ids
    ]
    model_inputs["labels"] = labels_ids
    return model_inputs


print("Preprocessing datasets...")
train_tokenized = train_dataset.map(preprocess, batched=True, remove_columns=train_dataset.column_names)
val_tokenized = val_dataset.map(preprocess, batched=True, remove_columns=val_dataset.column_names)

clear_memory()

In [ ]:
print(train_tokenized)

In [ ]:
print("Pad token ID:", tokenizer.pad_token_id)
print("Vocab size:", tokenizer.vocab_size)


#### **Data Collator (Dynamic Padding)**

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

#### **Load ROUGE for Evaluation**

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = [[(token if token != -100 else tokenizer.pad_token_id) for token in label] for label in labels]
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=predictions, references=labels, use_stemmer=True)
    return {k: round(v * 100, 2) for k, v in result.items()}

#### **Training Arguments**

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-news-summarizer",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,  # Small for Colab memory
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir='./logs',
)


#### **Trainer Setup**

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#### **Train the Model**

In [ ]:
clear_memory()
trainer.train()

#### **Evaluatin of Model**

In [ ]:
metrics = trainer.evaluate()
print(metrics)

#### **Test the Model on an article**

In [ ]:
import re

def clean_and_merge_article(article):
    # Step 1: Clean article text
    article = re.sub(r"\s+", " ", article.strip())  # collapse spaces & newlines
    article = article.replace(" ,", ",").replace(" .", ".")  # fix space before punctuation

    # Step 2: Summarize using your model
    inputs = tokenizer(article, return_tensors="pt", max_length=1024, truncation=True).to(model.device)
    summary_ids = model.generate(**inputs, max_length=142, min_length=56, length_penalty=2.0, num_beams=4)
    raw_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Step 3: Merge summary into one sentence
    summary = re.sub(r'\s+', ' ', raw_summary.strip())
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', summary)
    sentences = [s.strip(" .") for s in sentences if s.strip()]

    if not sentences:
        return ""
    if len(sentences) == 1:
        return sentences[0] + "."

    merged = ", ".join(sentences[:-1]) + " and " + sentences[-1]
    return merged.strip() + "."


In [ ]:
article = """
          President Bola Tinubu has paid tribute to a former Minister of Agriculture and ex-Peoples Democratic Party’s National Chairman, Chief Audu Ogbeh, describing him as a patriot who believed deeply in the nation’s potential.The family of the former minister announced his death in a statement earlier on Saturday.In a statement on Saturday by his Special Adviser on Information and Strategy, Bayo Onanuga, the President said the late politician was “always ready with facts and figures to support his propositions” and “a man of strong convictions who spoke the truth as he saw it.”The President added that Ogbeh’s contributions to Nigeria’s agricultural development, political stability, and democratic growth “will remain indelible in the country’s history.”Tinubu extended condolences to the government and people of Benue State, as well as Ogbeh’s family, friends, and associates, praying “the Almighty God will receive his soul and comfort his family at this difficult time.”

"""

In [ ]:
clean = clean_and_merge_article(article)
print(clean)